DSC160 Data Science and the Arts - Twomey - Spring 2020 - [dsc160.roberttwomey.com](http://dsc160.roberttwomey.com)

## Dimensional Reduction

This notebook explores techniques for dimensional reduction and visualization for high dimensional data (n>2) using python and jupyter. Dimensional reduction is a key problem in working with feature vectors and high dimensional data: how do we plot or reduce data from some higher dimensional space (n > 2) to a manageable lower dimensional space. 

For our purposes, an audio feature vector could contain 10-20 points (zero crossing rate, spectral centroid, MFCCs 0-20, etc.), or a visual feature vector could be extracted from an image using a pretrained Convolutional Neural Network.

Implements the following techniques:
- Principal Component Analysis (PCA) https://scikit-learn.org/stable/modules/decomposition.html#pca
- Multi-Dimensional Scaling (MDS)
- t-Distributed Stochastic Neighbor Embedding (t-SNE) https://lvdmaaten.github.io/tsne/
- Uniform Manifold Approximation and Projection (UMAP) for Dimension Reduction [https://arxiv.org/abs/1802.03426](https://arxiv.org/abs/1802.03426)

## PCA

Principal Component Analysis (PCA) - (from [wikipedia](https://en.wikipedia.org/wiki/Principal_component_analysis))

Given a collection of points in two, three, or higher dimensional space, a "best fitting" line can be defined as one that minimizes the average squared distance from a point to the line. The next best-fitting line can be similarly chosen from directions perpendicular to the first. Repeating this process yields an orthogonal basis in which different individual dimensions of the data are uncorrelated. These basis vectors are called principal components, and several related procedures principal component analysis (PCA). 

PCA is mostly used as a tool in exploratory data analysis and for making predictive models. It is often used to visualize genetic distance and relatedness between populations. PCA is either done in the following 2 steps:

1. calculating the data covariance (or correlation) matrix of the original data
2. performing eigenvalue decomposition on the covariance matrix

PCA is a linear dimensional reduction.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
digits = load_digits()
print(digits.DESCR)

The digits dataset is a collection of low res (8x8) grayscale images of handwritten digits. It is a common dataset for image classification and handwriting recognition research. 

To work with this data for a PCA analysis, we will first normalize the scale of the image data (divide by 255 to normalize from `0.0` to `1.0`). We will also grab the labels for the digit data. 

In [ ]:
X = digits.data / 255.0
y = digits.target
print(X.shape, y.shape)

Lets store the digits data in a Pandas DataFrame

In [ ]:
feat_cols = [ 'pixel'+str(i) for i in range(X.shape[1]) ]
df = pd.DataFrame(X,columns=feat_cols)
df['y'] = y
df['label'] = df['y'].apply(lambda i: str(i))
# X, y = None, None
print('Size of the dataframe: {}'.format(df.shape))

In [ ]:
# For reproducability of the results
np.random.seed(42)
rndperm = np.random.permutation(df.shape[0])

In [ ]:
plt.gray()
fig = plt.figure( figsize=(16,7) )
for i in range(0,15):
    ax = fig.add_subplot(3,5,i+1, title="Digit: {}".format(str(df.loc[rndperm[i],'label'])) )
    ax.matshow(df.loc[rndperm[i],feat_cols].values.reshape((8,8)).astype(float))
plt.show()

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(df[feat_cols].values)
df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df.loc[rndperm,:],
    legend="full",
    alpha=0.9
)
plt.show()

In [ ]:
# ax = plt.figure(figsize=(16,10)).gca(projection='3d')
# ax.scatter(
#     xs=df.loc[rndperm,:]["pca-one"], 
#     ys=df.loc[rndperm,:]["pca-two"], 
#     zs=df.loc[rndperm,:]["pca-three"], 
#     c=df.loc[rndperm,:]["y"], 
#     cmap='tab10'
# )
# ax.set_xlabel('pca-one')
# ax.set_ylabel('pca-two')
# ax.set_zlabel('pca-three')
# plt.show()

## Multi-Dimensional Scaling

Multidimensional scaling (MDS) is a means of visualizing the level of similarity of individual cases of a dataset. MDS is used to translate "information about the pairwise 'distances' among a set of n objects or individuals" into a configuration of n points mapped into an abstract Cartesian space. 

It is a form of non-linear dimensionality reduction.
(from [wikipedia](https://en.wikipedia.org/wiki/Multidimensional_scaling))


In [ ]:
from sklearn.datasets import load_digits
from sklearn.manifold import MDS

In [ ]:
# digits = load_digits()
# X = digits.data / 255.0
# y = digits.target

In [ ]:
X.shape

In [ ]:
X[0]

In [ ]:
embedding = MDS(n_components=2)

In [ ]:
X_transformed = embedding.fit_transform(X[:100])
# X_transformed = embedding.fit_transform(X)

In [ ]:
X_transformed.shape

In [ ]:
X_transformed[:,0]

In [ ]:
small_df = pd.DataFrame(X_transformed)
small_df['y'] = y[:len(X_transformed)]
# small_df

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x=X_transformed[:,0], y=X_transformed[:,1],
    hue='y',
    palette=sns.color_palette("hls", 10),
    data=small_df,
    legend="full",
    alpha=0.9
)
plt.show()

## t-SNE

T-distributed Stochastic Neighbor Embedding (t-SNE) is a machine learning algorithm for visualization developed by Laurens van der Maaten and Geoffrey Hinton. It is a nonlinear dimensionality reduction technique well-suited for embedding high-dimensional data for visualization in a low-dimensional space of two or three dimensions. Specifically, it models each high-dimensional object by a two- or three-dimensional point in such a way that similar objects are modeled by nearby points and dissimilar objects are modeled by distant points with high probability.

The t-SNE algorithm comprises two main stages. First, t-SNE constructs a probability distribution over pairs of high-dimensional objects in such a way that similar objects have a high probability of being picked while dissimilar points have an extremely small probability of being picked. Second, t-SNE defines a similar probability distribution over the points in the low-dimensional map, and it minimizes the Kullback–Leibler divergence (KL divergence) between the two distributions with respect to the locations of the points in the map. (from [wikipedia](https://en.wikipedia.org/wiki/Multidimensional_scaling))

good online demo to explore the concept: https://distill.pub/2016/misread-tsne/

In [ ]:
from sklearn.manifold import TSNE
import time

In [ ]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(X)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
len(tsne_results)

In [ ]:
len(df)

In [ ]:
df['tsne-2d-one'] = tsne_results[:,0]
df['tsne-2d-two'] = tsne_results[:,1]

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df,
    legend="full",
    alpha=0.9
)
plt.show()

In [ ]:
plt.figure(figsize=(16,7))
ax1 = plt.subplot(1, 2, 1)
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df,
    legend="full",
    alpha=0.3,
    ax=ax1
)
ax2 = plt.subplot(1, 2, 2)
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df,
    legend="full",
    alpha=0.3,
    ax=ax2
)
plt.show()

## UMAP


Uniform Manifold Approximation and Projection (UMAP) is a dimension reduction
technique that can be used for visualisation similarly to t-SNE, but also for
general non-linear dimension reduction. The algorithm is founded on three
assumptions about the data

1. The data is uniformly distributed on Riemannian manifold;
2. The Riemannian metric is locally constant (or can be approximated as such);
3. The manifold is locally connected.

From these assumptions it is possible to model the manifold with a fuzzy
topological structure. The embedding is found by searching for a low dimensional
projection of the data that has the closest possible equivalent fuzzy
topological structure.

The details for the underlying mathematics can be found in
[their paper on ArXiv](https://arxiv.org/abs/1802.03426).

In [ ]:
# run once
# !pip uninstall -y umap --user
!pip install umap-learn --user

### UMAP with Iris Data

Below is the UMAP example with iris data from https://umap-learn.readthedocs.io/en/latest/basic_usage.html#iris-data

In [ ]:
import numpy as np
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

In [ ]:
# seaborn for plotting, we can also use bokeh
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

In [ ]:
iris = load_iris()
print(iris.DESCR)

In [ ]:
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = pd.Series(iris.target).map(dict(zip(range(3),iris.target_names)))
sns.pairplot(iris_df, hue='species');

(bad workaround, see here: https://github.com/lmcinnes/umap/issues/24)

In [ ]:
import umap.umap_ as umap

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding = reducer.fit_transform(iris.data)
embedding.shape

In [ ]:
plt.scatter(embedding[:, 0], embedding[:, 1], c=[sns.color_palette()[x] for x in iris.target])
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the Iris dataset', fontsize=24);

### UMAP with DIGITS data

Below is the UMAP example with DIGITS data https://umap-learn.readthedocs.io/en/latest/basic_usage.html#digits-data

In [ ]:
digits = load_digits()
print(digits.DESCR)

In [ ]:
fig, ax_array = plt.subplots(20, 20)
axes = ax_array.flatten()
for i, ax in enumerate(axes):
    ax.imshow(digits.images[i], cmap='gray_r')
plt.setp(axes, xticks=[], yticks=[], frame_on=False)
plt.tight_layout()

In [ ]:
digits_df = pd.DataFrame(digits.data[:,:10])
digits_df['digit'] = pd.Series(digits.target).map(lambda x: 'Digit {}'.format(x))
sns.pairplot(digits_df, hue='digit', palette='Spectral');

#### do the UMAP reduction

In [ ]:
reducer = umap.UMAP(random_state=42)
reducer.fit(digits.data)

In [ ]:
embedding = reducer.transform(digits.data)
# Verify that the result of calling transform is
# idenitical to accessing the embedding_ attribute
assert(np.all(embedding == reducer.embedding_))
embedding.shape

In [ ]:
plt.figure(figsize=(16, 10), dpi=80)
plt.scatter(embedding[:, 0], embedding[:, 1], c=digits.target, cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
plt.title('UMAP projection of the Digits dataset', fontsize=24);

view these results with bokeh and tooltips

In [ ]:
from io import BytesIO
from PIL import Image
import base64

In [ ]:
def embeddable_image(data):
    img_data = 255 - 15 * data.astype(np.uint8)
    image = Image.fromarray(img_data, mode='L').resize((64, 64), Image.BICUBIC)
    buffer = BytesIO()
    image.save(buffer, format='png')
    for_encoding = buffer.getvalue()
    return 'data:image/png;base64,' + base64.b64encode(for_encoding).decode()

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10

output_notebook()

In [ ]:
digits_df = pd.DataFrame(embedding, columns=('x', 'y'))
digits_df['digit'] = [str(x) for x in digits.target]
digits_df['image'] = list(map(embeddable_image, digits.images))

datasource = ColumnDataSource(digits_df)
color_mapping = CategoricalColorMapper(factors=[str(9 - x) for x in digits.target_names],
                                       palette=Spectral10)

plot_figure = figure(
    title='UMAP projection of the Digits dataset',
    plot_width=600,
    plot_height=600,
    tools=('pan, wheel_zoom, box_zoom, reset')
)

plot_figure.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <img src='@image' style='float: left; margin: 5px 5px 5px 5px'/>
    </div>
    <div>
        <span style='font-size: 16px; color: #224499'>Digit:</span>
        <span style='font-size: 18px'>@digit</span>
    </div>
</div>
"""))

plot_figure.circle(
    'x',
    'y',
    source=datasource,
    color=dict(field='digit', transform=color_mapping),
    line_alpha=0.6,
    fill_alpha=0.6,
    size=4
)
show(plot_figure)

### References
- UMAP reference here https://umap-learn.readthedocs.io/en/latest/basic_usage.html#digits-data